In [8]:
import numpy as np
from tenpy.networks.mps import TransferMatrix

In [13]:
def pollmann_turner(self, Op = 'inversion', charge='all'):
    """ Obtains the matrices of the action of a symmetry 'Op' on the right Schmidt states:
    
        Op_R |i; R> = U_{i, j} |j; R>
        
    Op is either: 
        - list of single site matrices to be applied to the sites
        - 'inversion' : inverts orbital ordering
        - 'time_reversal': complex conjugation
        - an MPS representing Op psi
    
    Returns:
        Tr(U U^*)/chi   (pm 1 for inv and time reversal)
        U
        q, the charge sector of dominant eigenvalue
    """
    self.convert_form('B')
    if Op=='inversion':
        psi_inv = self.copy().spatial_inversion() #Form spatial reflection of the WF
    elif Op=='time_reversal':
        psi_inv = self.copy()
        for i in range(psi_inv.L):
            psi_inv.setB(i, psi_inv.getB(i).iconj() )
    elif type(Op)==iMPS:
        psi_inv = Op
    else:
        if type(Op)!=list:
            Op = list(Op)
            
        psi_inv = self.copy()
        for i in range(psi_inv.L):
            psi_inv.setB(i, npc.tensordot(self.get(Op, i), psi_inv.getB(i), axes = [[1], [0]] ) )
            
            
    T = TransferMatrix(psi_inv, self) #Represents generalized T matrix
    
    
    #T does diagonalization by charge sector, so we must loop over sectors
    if charge =='all':
        q_sectors = T.qt_ind_dict.keys()
    else:
        q_sectors = charge
        
    eta_max = 0	#largest eta (eigenvalue) we've found so far
    
    for charge_sector in q_sectors:
        try:
            eta, G = T(charge_sector).eigenvectors(num_ev=1, verbose=False) #Find dominant eigenvector
            print("Q, |eta|", charge_sector, np.abs(eta))
        except:
            print("Sector", charge_sector, "did not converge.")
            
            raise
            #eta = [0.]
            #G = [None]
            
            #If this is getting to be a problem, we can do dense diag as below.
            #m = matvec_to_array(T(charge_sector))
            #eta, G =  np.linalg.eig(m)
            #perm = np.argsort(-np.abs(eta))
            #eta = eta[perm]
            #print eta
            #G = [ T(charge_sector).to_npc_array(G[:, 0]) ]
        
        #eta = eta[0] #eigenvalue
        #G = G[0] #This should be the 'U_I/chi' object of Pollmann-Turner
        
        if np.abs(eta) > np.abs(eta_max): #Or we could just check if it is 1...?
            if np.abs(eta_max - 1.) < 10**(-8):
                print("WARNING: T-mat has degeneracy at eta = 1.")
            U = G[0]
            eta_max = eta
            q_max = charge_sector
            #print G.shape[0]*npc.tensordot(G, G.conj(), [[1], [1]]) #This should be identity
            #print "eta, ph, G:", eta, ph, G
            
    U = U*np.sqrt(U.shape[0])
    print("Using charge sector", q_max, "|eta| = ", np.abs(eta_max))
    print("| U.Ud - 1|", npc.norm(npc.tensordot(U, U.conj(), axes = [[1], [1]]) - npc.eye_like(U)))
    s = self.s[-1]
    print("|Us - sU|", npc.norm(U.scale_axis(s, 0) - U.scale_axis(s, 1)))
    p = npc.inner(U, U.conj(), [[0, 1], [1, 0]])/U.shape[0]
            

    return p, U, q_max

# Generate an MPS

In [3]:
# import the model file we designed
"""
    Here we want to clarify that the 'kitaev_ladder_snake1.py' file
    is different from the 'kitaev_ladder_snake.py' in that the *1 file 
    uses the original representation of xyz,
    i.e. Jz stands for the sigmazsigmaz interactions which starts from
    the first pair of neighbors; Jx being the second pair; and Jy being
    the third-nearest-neighbors.
    
    The purpose of designing so many different 1D correspondences is 
    to serve for different phases.
    For example, when the Jx interaction is very weak, the optimal 
    choice is to make Jx as the third-nearest neighbors; when Jy is 
    weak the similar case holds.
"""
from kitaev_ladder_snake1 import run_atomic, save_after_run, finite_scaling, load_data, fDMRG_KL

# the folder name for storing data
folder_prefix = 'snake/'

# generate the function for running and storing the data in the folder we want
run_save = save_after_run(run_atomic, folder_prefix=folder_prefix)

In [4]:
Jx = 1.75
Jy = 0.25
Jz = 1.0

In [5]:
chi_list = [2**k for k in range(3, 8)]
initial_psi = None

for chi in chi_list:
    result = run_atomic(Jx=Jx, Jy=Jy, Jz=Jz, chi=chi, initial_psi=initial_psi, L=4, bc='periodic', bc_MPS='infinite')
    psi = result['psi']
    initial_psi = psi.copy()
    pass

psi = initial_psi.copy()

Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  8 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sweeps...
Reading 'start_env'=10 for config TwoSiteDMRGEngine
Updating environment
..........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 408
Energy = -0.2396616069357673, S = nan, norm_err = 4.6e-09
Current memory usage 151.4 MB, time elapsed: 6.2 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.4513e-09, max_E_trunc = 4.1108e-09
MPS bond dimensions: [16, 16, 16, 16, 16, 16, 16, 16]
Updating environment
.....
sweep 20, age = 648
Energy = -0.2396616074227431, S = nan, norm_err = 8.8e-10
Current memory usage 151.4 MB, time elapsed: 9.9 s
Delta E = -4.8698e-11, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.1122e-09, max_E_trunc = 1.6254e-09
MPS bond dimensions: [16, 16, 16, 16, 16, 16, 16, 16]

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


Before the canonicalization:
Bond dim =  [26, 32, 26, 32, 26, 32, 26, 32]
Canonicalizing...
The norm is:  1.0
The overlap is:  (1.0000000000000118-1.265249761192137e-16j)
After the canonicalization:
Bond dim =  [26, 32, 26, 32, 26, 32, 26, 32]
Computing properties
Optimizing
Loop for chi=32 done.
============================== END ==============================
Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  64 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for conf

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


Before the canonicalization:
Bond dim =  [28, 36, 28, 36, 28, 36, 28, 36]
Canonicalizing...
The norm is:  1.0
The overlap is:  (0.9999999999999953+3.8917019812148477e-16j)
After the canonicalization:
Bond dim =  [28, 36, 28, 36, 28, 36, 28, 36]
Computing properties
Optimizing
Loop for chi=64 done.
============================== END ==============================
Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  128 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for co

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


In [6]:
psi_test = psi.copy()

In [14]:
pollmann_turner(psi_test)

AttributeError: 'TransferMatrix' object has no attribute 'qt_ind_dict'

In [ ]:
import tenpy
tenpy.version.full_version